In [8]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import numpy as np
import pandas as pd


def loadDataSet():
    '''创建一个用于测试的简单的数据集'''

    data = pd.read_csv("train.csv")
    data = data.sample(1000)
#     import copy
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data)) # 将数据扩大8倍
    print(data.shape)
    def gerDoodAndBad(score):
        ans = []

        # math judge
        if score['math score'] > 80:
            ans.append('MG')
        elif score['math score'] < 60:
            ans.append('MB')

        # reading judge
        if score['reading score'] > 85:
            ans.append('RG')
        elif score['reading score'] < 50:
            ans.append('RB')

        # writing judge
        if score['writing score'] > 85:
            ans.append('WG')
        elif score['writing score'] < 50:
            ans.append('WB')

        if score['test preparation course'] == 1:
            ans.append("preparation")

        return ans

    data['Apriori'] = data.apply(lambda x: gerDoodAndBad(x), axis=1)
#     print(data.head())

    return np.array(data['Apriori'])


def loadTest():
    import json

    f = open("para.json", 'r')
    para = json.loads(f.read())
    
    testD = pd.read_csv('testData')
    testD = testD.head(para['testDataSize'])
    testD.shape
    npt = np.array(testD)
    npt
    return npt


# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))
    ssCnt = {}
    for tid in D:  # 对于每一条transaction
        for can in Ck:  # 对于每一个候选项集can，检查是否是transaction的一部分 # 即该候选can是否得到transaction的支持
            flag = True
            for i in can:
                if i not in tid:
                    flag = False
                    
            if flag:
                ssCnt[can] = ssCnt.get(can, 0) + 1
                
#             if can.issubset(tid):
#                 ssCnt[can] = ssCnt.get(can, 0) + 1
    numItems = float(len(D))
    # print("ssCnt is",ssCnt)
    retList = []
    supportData = {}
    for key in ssCnt:
        support = ssCnt[key] / numItems  # 每个项集的支持度
        if support >= minSupport:  # 将满足最小支持度的项集，加入retList
            retList.insert(0, key)
        supportData[key] = support  # 汇总支持度数据
    return retList, supportData


def aprioriGen(Lk, k):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''
    retList = set()
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i + 1, lenLk):
            
            L1 = Lk[i]
            L2 = Lk[j]
            cnt =0
            for m in L1:
                if m in L2:
                    cnt+=1
            if cnt == k-2:
                retList.add(Lk[i] | Lk[j])
    return retList


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    C1 = createC1(
        dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]

    D = [set(var) for var in dataSet]  # 集合化数据集
    L1, suppData = scanD(D, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素
    L = [L1]  # 最初的L1中的每个项集含有一个元素，新生成的
    # print()
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t = time.time()
        Ck = aprioriGen(L[k - 2], k)
        print(f'gen coast:{time.time() - t:.8f}s')
        
        t = time.time()
        Lk, supK = scanD(D, Ck, minSupport) # 筛选最小支持度的频繁项集
        print(f'scan coast:{time.time() - t:.8f}s')
        # print("iter is ")
        # print(Ck)
        # print(Lk)
        # print()
        suppData.update(supK)  # 将新的项集的支持度数据加入原来的总支持度字典中
        L.append(Lk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L, suppData  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList




In [9]:
myDat = loadTest()  # 导入数据集
myDat.shape

(1000, 23)

In [10]:

import time
t = time.time()

L, suppData = apriori(myDat, 0.3)  # 选择频繁项集
# print(u"频繁项集L：", suppData)
# print(u"所有候选项集的支持度信息：", suppData)
print(f'花费的时间为:{time.time() - t:.8f}s')

22
iter is  2
gen coast:0.00000000s
120
scan coast:0.12128401s
iter is  3
gen coast:0.00804901s
529
scan coast:0.55353427s
iter is  4
gen coast:0.15910244s
1506
scan coast:1.58464718s
iter is  5
gen coast:0.48731828s
2911
scan coast:3.23005486s
iter is  6
gen coast:1.41848564s
3914
scan coast:4.83819628s
iter is  7
gen coast:1.75707054s
3675
scan coast:4.81357265s
iter is  8
gen coast:1.03155327s
2369
scan coast:3.23273921s
iter is  9
gen coast:0.32773948s
1001
scan coast:1.62619948s
iter is  10
gen coast:0.02393150s
250
scan coast:0.39178705s
iter is  11
gen coast:0.00000000s
28
scan coast:0.04792452s
iter is  12
gen coast:0.00000000s
0
scan coast:0.00000000s
花费的时间为:25.72822475s


In [ ]:
rules = generateRules(L, suppData, minConf=0.99)
# print('rules:\n', rules)

## CBE Apriori二进制编码算法

In [2]:
# frozenset() 返回一个冻结的集合，冻结后集合不能再添加或删除任何元素。
import numpy as np
import pandas as pd



class node:
    def __int__(self):
        self.items = []
        self.key = 0
        self.sup = 0
    def show(self):
        print(self.items, bin(self.key), end=" ")
        try:
            print(self.sup)
        except:
            print()
            pass


def ListToNode(items):
    nodeT = node()
    nodeT.key =0
    nodeT.items = items
    for index,i in enumerate(defaultSL): # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index -1)
    return nodeT


def keyToList(key):
    ans =[]
    global defaultSL
    for i in range(1,len(defaultSL) +1):
        if key  & (1 << (len(defaultSL)) - i) == (1 << (len(defaultSL)) - i): # 含有第i个
            ans.append(defaultSL[i-1])
    return ans


def loadDataSet():
    '''创建一个用于测试的简单的数据集'''

    data = pd.read_csv("train.csv")
    data = data.sample(1000)
#     import copy
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data))
#     data = data.append(copy.deepcopy(data)) # 将数据扩大8倍
    print(data.shape)
    def gerDoodAndBad(score):
        ans = []

        # math judge
        if score['math score'] > 80:
            ans.append('MG')
        elif score['math score'] < 60:
            ans.append('MB')

        # reading judge
        if score['reading score'] > 85:
            ans.append('RG')
        elif score['reading score'] < 50:
            ans.append('RB')

        # writing judge
        if score['writing score'] > 85:
            ans.append('WG')
        elif score['writing score'] < 50:
            ans.append('WB')

        if score['test preparation course'] == 1:
            ans.append("preparation")

        return ans

    data['Apriori'] = data.apply(lambda x: gerDoodAndBad(x), axis=1)
#     print(data.head())

    return np.array(data['Apriori'])

def loadTest():
    import json

    f = open("para.json", 'r')
    para = json.loads(f.read())
    
    testD = pd.read_csv('testData')
    testD = testD.head(para['testDataSize'])
    testD.shape
    npt = np.array(testD)
    npt
    return npt

# 返回只有单个元素的候选集
def createC1(dataSet):
    '''
        构建初始候选项集的列表，即所有候选项集只包含一个元素，
        C1是大小为1的所有候选项集的集合
    '''
    C1 = []
    for transaction in dataSet:
        for item in transaction:
            if [item] not in C1:
                C1.append([item])
    C1.sort()
    # return map( frozenset, C1 )
    # return [var for var in map(frozenset,C1)]
    return [frozenset(var) for var in C1]


def scanD(D, Ck, minSupport):
    '''
        计算Ck中的项集在数据集合D(记录或者transactions)中的支持度,
        返回满足最小支持度的项集的集合，和所有项集支持度信息的字典。
    '''
    print(len(Ck))

    for i in Ck:
        i.sup =0
        for item in D:
            if i.key & item.key == i.key: #与运算判断子集
                i.sup +=1
    ans =[]
    length = len(D)
    for i in Ck:
        i.sup = i.sup / length
        if i.sup >= minSupport:
            ans.append(i)
            # i.show()
    return ans



def aprioriGen(Lk, k):  # Aprior算法
    '''
        由初始候选项集的集合Lk生成新的生成候选项集，
        k表示生成的新项集中所含有的元素个数
        注意其生成的过程中，首选对每个项集按元素排序，然后每次比较两个项集，只有在前k-1项相同时才将这两项合并。这样做是因为函数并非要两两合并各个集合，那样生成的集合并非都是k+1项的。在限制项数为k+1的前提下，只有在前k-1项相同、最后一项不相同的情况下合并才为所需要的新候选项集。
    '''

    global twoDif

    retList = []
    lenLk = len(Lk)
    nowSet = set()
    for i in range(lenLk):
        for j in range(i + 1, lenLk):

            a = Lk[i]
            b = Lk[j]
            t = a.key ^ b.key
            #要注意候选集以及去重的问题
            if t in twoDif:

                
                tkey = a.key | b.key
                if tkey not in nowSet:
                    nodeT = node()
                    nodeT.key = tkey
                    nowSet.add(nodeT.key)
                    # a.show()
                    # b.show()
#                     nodeT.items = keyToList(nodeT.key)  #生成新的items
                    retList.append(nodeT)
                else:
                    pass

    return retList


def apriori(dataSet, minSupport=0.5):
    """
    该函数为Apriori算法的主函数，按照前述伪代码的逻辑执行。Ck表示项数为k的候选项集，最初的C1通过createC1()函数生成。Lk表示项数为k的频繁项集，supK为其支持度，Lk和supK由scanD()函数通过Ck计算而来。
    :param dataSet:
    :param minSupport:
    :return:
    """
    # C1 = createC1(dataSet)  # 构建初始候选项集C1  [frozenset({1}), frozenset({2}), frozenset({3}), frozenset({4}), frozenset({5})]
    global defaultSL
    global defaultS

#     print(defaultSL)
    C1 = [ListToNode([i]) for i in defaultSL]


    # D = [set(var) for var in dataSet]  # 集合化数据集

    F1 = scanD(dataSet, C1, minSupport)  # 构建初始的频繁项集，即所有项集只有一个元素

    # print()
    L = [F1]
    k = 2  # 项集应该含有2个元素，所以 k=2

    while (len(L[k - 2]) > 0):
        print("iter is ", k)
        t= time.time()
        Ck = aprioriGen(L[k - 2], k) # 计算候选集
        print(f'gen coast:{time.time() - t:.8f}s')
        t = time.time()
        
        Fk= scanD(dataSet, Ck, minSupport) # 筛选最小支持度的频繁项集
        print(f'scan coast:{time.time() - t:.8f}s')

        L.append(Fk)  # 将符合最小支持度要求的项集加入L
        k += 1  # 新生成的项集中的元素个数应不断增加
    return L  # 返回所有满足条件的频繁项集的列表，和所有候选项集的支持度信息


def calcConf(freqSet, H, supportData, brl, minConf=0.7):  # 规则生成与评价
    '''
        计算规则的可信度，返回满足最小可信度的规则。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中所有的元素
        supportData(dic):频繁项集中所有元素的支持度
        brl(tuple):满足可信度条件的关联规则
        minConf(float):最小可信度
    '''
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print(freqSet - conseq, '-->', conseq, 'conf:', conf)
            brl.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH


def rulesFromConseq(freqSet, H, supportData, brl, minConf=0.7):
    '''
        对频繁项集中元素超过2的项集进行合并。
        freqSet(frozenset):频繁项集
        H(frozenset):频繁项集中的所有元素，即可以出现在规则右部的元素
        supportData(dict):所有项集的支持度信息
        brl(tuple):生成的规则
    '''
    m = len(H[0])
    if len(freqSet) > m + 1:  # 查看频繁项集是否足够大，以到于移除大小为 m的子集，否则继续生成m+1大小的频繁项集
        Hmp1 = aprioriGen(H, m + 1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, brl, minConf)  # 对于新生成的m+1大小的频繁项集，计算新生成的关联规则的右则的集合
        if len(Hmp1) > 1:  # 如果不止一条规则满足要求（新生成的关联规则的右则的集合的大小大于1），进一步递归合并，
            # 这样做的结果就是会有“[1|多]->多”(右边只会是“多”，因为合并的本质是频繁子项集变大，
            # 而calcConf函数的关联结果的右侧就是频繁子项集）的关联结果
            rulesFromConseq(freqSet, Hmp1, supportData, brl, minConf)


def generateRules(L, supportData, minConf=0.7):
    '''
        根据频繁项集和最小可信度生成规则。
        L(list):存储频繁项集
        supportData(dict):存储着所有项集（不仅仅是频繁项集）的支持度
        minConf(float):最小可信度
    '''
    bigRuleList = []
    for i in range(1, len(L)):
        for freqSet in L[i]:  # 对于每一个频繁项集的集合freqSet
            H1 = [frozenset([item]) for item in freqSet]
            if i > 1:  # 如果频繁项集中的元素个数大于2，需要进一步合并，这样做的结果就是会有“[1|多]->多”(右边只会是“多”，
                # 因为合并的本质是频繁子项集变大，而calcConf函数的关联结果的右侧就是频繁子项集），的关联结果
                rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
            else:
                calcConf(freqSet, H1, supportData, bigRuleList, minConf)

    sorted(bigRuleList)
    return bigRuleList





In [3]:
defaultS = set()  # 存储所有单项的集合
myDat = loadTest()  # 导入数据集
for i in myDat:
    for j in i:
        if j not in defaultS:
            defaultS.add(j)
defaultSL = list(defaultS)
defaultSL = sorted(defaultSL) # 把所有单项计算出来并排序，形成默认顺序，方便后面进行二进制编码
print(defaultSL, len(defaultSL))
Items = []



#对项集进行二进制编码
for items in myDat:
    nodeT = node()
    nodeT.key =0
    nodeT.items = items
    for index,i in enumerate(defaultSL): # 进行编码
        if i in items:
            nodeT.key += 1 << (len(defaultSL) - index -1)
    Items.append(nodeT)
    # print(items, bin(nodeT.key))


['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'k', 'l', 'm', 'n', 'o', 'p', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y'] 22


得到默认编码序列['MB', 'MG', 'RB', 'RG', 'WB', 'WG', 'preparation']

In [4]:

twoDif =set() # 计算仅仅两个不同时的二进制集合
length = len(defaultSL)
for i in range(length):
    for j in range(length):
        if i != j:
            t=0
#             print(i,j)
            t = 1<<(i) 
            t+=1<<(j)
#             print(bin(t))
            twoDif.add(t)
for i in twoDif:
    print(bin(i))

0b11000000000
0b1000000001
0b10000000001
0b11
0b100000000001
0b101
0b1000000000001
0b10000000000001
0b100000000000001
0b1001
0b1000000000000001
0b10000000000000001
0b10000000000000000001
0b1000000000000000001
0b1010
0b1100
0b1000000001000
0b10001
0b10010
0b100000000000000010
0b10100
0b100000000000100
0b1000000000000100
0b10000000000000100
0b100000000000000100
0b1000000000000000100
0b10000000000000000100
0b1000000010
0b100000000000000000100
0b1000000000000000000100
0b11000
0b100000000000000000001
0b1000000000000000000001
0b100001
0b110
0b100010
0b100100
0b1000000100
0b1000100000
0b10000100000
0b101000
0b100000000001000
0b1000000000001000
0b10000000010
0b10000000000001000
0b100000000000001000
0b1000000000000001000
0b10000000000000001000
0b100000000000000001000
0b1000000000000000001000
0b110000
0b10000000100
0b1100000000000000
0b10000000000000000010
0b100000000000000000010
0b1000000000000000000010
0b1000001000
0b1001000000
0b1000001
0b1000010
0b10001000000
0b1000100
0b100001000000
0b10000

In [5]:
len(Items)

1000

In [6]:
import time
t = time.time()

L = apriori(Items, 0.3)  # 选择频繁项集
# print(u"频繁项集L：")
# for li in L:
#     for i in li:
#         i.show()
print(f'total coast:{time.time() - t:.8f}s')



22
iter is  2
gen coast:0.00000000s
120
scan coast:0.09599853s
iter is  3
gen coast:0.00000000s
529
scan coast:0.28920817s
iter is  4
gen coast:0.06396151s
1506
scan coast:0.79700947s
iter is  5
gen coast:0.32088709s
2911
scan coast:1.43136430s
iter is  6
gen coast:0.71263957s
3914
scan coast:1.54763031s
iter is  7
gen coast:0.80394101s
3675
scan coast:1.46617723s
iter is  8
gen coast:0.44769478s
2369
scan coast:0.94741082s
iter is  9
gen coast:0.07991552s
1001
scan coast:0.45747447s
iter is  10
gen coast:0.00794983s
250
scan coast:0.10396671s
iter is  11
gen coast:0.00000000s
28
scan coast:0.00796056s
iter is  12
gen coast:0.00000000s
0
scan coast:0.00000000s
total coast:9.60652113s


### 这里对比原生的Apriori算法，可以看出答案是一样的。正确

### 关于运行速度对比
- 如果采用原生的教学数据集看不出太大差别，因为数据集很小。
- 但是如果采用稍微正常一点的不是很稠密的数据集，不难看出。随着最小支持度的减小，二者的差别越来越大
- 这里的数据集采用mushroom

### 关于运行效率分析
这里的数据集特征，
- 总计8123条数据,23个特征
- 如果数据条数足够大的时候，时间主要集中在gen上，这时候二进制编码的效率就上来了
- 当特征太少的时候，二者跑不出太大区别，因为基本都是单项集scan和查找候选集都没有太大差别

### 结果

当数据集为500,23特征，支持度为0.3
- 经典Apriori：21.2s
- 二进制Apriori：5.8s

3000,23特征，支持度为0.3
- 经典Apriori：分钟了
- 二进制Apriori：30.2s




In [7]:
for i in L:
    print(len(i))

16
107
401
946
1484
1582
1142
541
156
23
1
0
